<a href="https://colab.research.google.com/github/dsibi/yandex_praktikum_da/blob/main/catering_market_research_in_moscow/catering_market_research_in_moscow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Исследование рынка общественного питания в Москве**

---

## Описание проекта

Вы решили открыть небольшое кафе в Москве. Оно оригинальное — гостей должны обслуживать роботы. Проект многообещающий, но дорогой. Вместе с партнёрами вы решились обратиться к инвесторам. Их интересует текущее положение дел на рынке — сможете ли вы снискать популярность на долгое время, когда все зеваки насмотрятся на роботов-официантов?

Вы — гуру аналитики, и партнёры просят вас подготовить исследование рынка. У вас есть открытые данные о заведениях общественного питания в Москве.

##Описание данных

Таблица rest_data:
*  id — идентификатор объекта;
*  object_name — название объекта общественного питания;
*  chain — сетевой ресторан;
*  object_type — тип объекта общественного питания;
*  address — адрес;
*  number — количество посадочных мест.

Импортируем все необходимые нам библиотеки, далее новые библиотеки будем добавлять сюда же

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import transforms
import numpy as np
import re
import requests
from io import BytesIO
pd.options.mode.chained_assignment = None

In [2]:
api_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Установим безлимитную ширину колонки при выводе, чтобы названия в колонках выводились полностью.

In [3]:
pd.set_option('display.max_colwidth', None)

##Шаг 1. Загрузим данные и подготовим их к анализу

---



###1.1. Загрузим файл с данными и изучим общую информацию

Чтение файлов

In [4]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
cafe=pd.read_csv('/content/drive/MyDrive/Edu/Data Analysis/9_Как рассказать историю с помощью данных/!Project/data/rest_data.csv')

In [ ]:
# cafe=pd.read_csv('/datasets/rest_data.csv')

In [ ]:
cafe.sample(10)

Заменим 'предприятие быстрого обслуживания' на 'бистро', а 'магазин (отдел кулинарии)' на 'кулинария' для более простой последующей обработки, а также приведем унификацию в названиях ресторанов:

In [ ]:
cafe['object_name'] = cafe['object_name'].str.lower()
cafe['object_name'] = cafe['object_name'].str.replace(r'[«».]*', '')
cafe['object_name'] = cafe['object_name'].str.replace(r'-', ' ')
cafe.replace({"object_type": {'предприятие быстрого обслуживания':'бистро',
                            'магазин (отдел кулинарии)': 'магазин'}}, inplace=True)

In [ ]:
cafe.sample(10)

Изучим общую информацию 

In [ ]:
df_list = [cafe]
for df in df_list:   
    print('Первые 5 строк датафрейма')
    print('-'*80)
    display(df.head(5))
    print('-'*80)
    print('Типы данных и общая информация')
    print(df.info())
    print('-'*80)
    print('Наличие дубликатов: {}'.format(df.duplicated().sum()))
    print('-'*80)
    print('Наличие пропусков')
    print('-'*80)
    print(df.isna().sum())
    print('Стат данные')
    print('-'*80)
    display(df.describe())

1. Названия столбцов: корректировка не требуется.
2. Типы данных: корректировка не требуется.
3. Дубликаты: отсутствуют.
4. Пропуски: отсутствуют.
5. Аномалии в данных: присутствуют записи с количеством посадочных мест равным нулю.

###1.2. Подготовим данные к анализу

Посчитаем количество заведений, где количество посадочных мест равно нулю:

In [ ]:
print(cafe[cafe.number == 0].shape[0])

Более 10 % датасета с записями, где количество посадочных мест равно нулю. Это достаточно большой показатель, посмотрим, что это за заведения:

In [ ]:
cafe.loc[cafe['number'] == 0].sample(n=10)

Выявили, что это всякого рода буфеты, закусочные, предприятия быстрого обслуживания, отделы кулинарии в магазинах, т.е. те заведения, где в принципе могут быть не предусмотрены посадочные места, поэтому ничего с этими записями делать не будем.

##Шаг 2. Анализ данных

---



###2.1. Соотношение видов объектов общественного питания (далее - ООП) по количеству

In [ ]:
cafe_by_object_type=cafe['object_type'].value_counts().reset_index()
cafe_by_object_type.columns = ['object_type','count']

In [ ]:
plt.figure(figsize=(8,6))
ax=sns.barplot(x='object_type', y="count", data=cafe_by_object_type, 
            order=cafe_by_object_type.sort_values('count', ascending = False).object_type,
                 color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right', size=14)
for p in ax.patches:
    ax.annotate(format(int(p.get_height()), ',').replace(',', ' '),
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 7), 
                   textcoords = 'offset points', size=14)
ax.axes.get_yaxis().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlabel(" ")
plt.title("Виды ООП по количеству заведений", size=18)
plt.tight_layout()

Почти половину ООП составляют кафе, за ними следуют столовые и рестораны. Совсем малое количество ООП по отделам кулинарий и закусочных.

Доминирование численности таких ООП как кафе может заключаться в большой популярности такого вида объекта как для посетителей, так и для владельцев этого вида ООП - первых привлекает хорошее сочетание цена-качество, а для вторых содержание и затраты меньшие чем, например, ресторанов.



###2.2. Соотношение сетевых и несетевых заведений по количеству

In [ ]:
cafe_by_chain = cafe.pivot_table(index=['object_type','chain'], values='id', aggfunc='count').reset_index()
cafe_by_chain.columns = ['object_type','chain','count']

In [ ]:
ax=sns.catplot(x='object_type', y='count', hue='chain', data=cafe_by_chain, kind='bar', height=4, aspect=4, 
            order=cafe_by_object_type.sort_values('count', ascending = False).object_type, palette="Paired", legend=False)
ax.set_xticklabels(rotation=50, horizontalalignment ='right', size=14)
for ax in ax.axes.ravel():
    for p in ax.patches:
        ax.annotate(format(int(p.get_height()), ',').replace(',', ' '),
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'center', 
                       xytext = (0, 7), 
                       textcoords = 'offset points', size=14)
plt.xlabel(" ")
plt.ylabel(" ")
plt.title("Соотношение сетевых и несетевых заведений", size=18)
L=plt.legend()
L.get_texts()[0].set_text('Сетевой ресторан')
L.get_texts()[1].set_text('Несетевой ресторан')

По всем ООП несетевые рестораны превышают сетевые, особенно ярко это видно по столовым.

Отсутствие сетевых столовых вероятнее всего вызвано тем, что обычно такие ООП Размещаются на территории предприятий/офисов/учреждений.

###2.3. Виды объектов общественного питания с характерным сетевым распространением

In [ ]:
cafe_chain_rate= cafe.pivot_table(index=['object_type'], columns='chain', values='id', aggfunc='count').reset_index()
cafe_chain_rate.columns = ['object_type','chain_count','not_chain_count']
cafe_chain_rate['total_count']=cafe_chain_rate['chain_count']+cafe_chain_rate['not_chain_count']
cafe_chain_rate['chain_rate']=cafe_chain_rate['chain_count']/cafe_chain_rate['total_count']*100

In [ ]:
plt.figure(figsize=(15,6))
ax=sns.barplot(x='object_type', y="chain_rate", data=cafe_chain_rate, 
            order=cafe_chain_rate.sort_values('chain_rate', ascending = False).object_type,
                 color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right', size=14)
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()/100
    x, y = p.get_xy() 
    ax.annotate(f'{height:.2%}', (x + width/2, (y + height*100)+.7), ha='center', size=14)
ax.axes.get_yaxis().set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlabel(" ")
plt.title("Доля ООП с сетевым распространением по их видам", size=18)
plt.tight_layout()

Характерными свойствами сетевого распространения являются предприятия быстрого обслуживания с долей сетевых ресторанов - 41,13 %, отделы кулинарии в магазинах - 28,57 %, рестораны и кафе - 23,81 % и 22,89 % соответственно.

###2.4. Характерные черты сетевых заведений по количеству посадочных мест

In [ ]:
def rainbow_text(x,y,ls,lc,**kw):
 
    t = plt.gca().transData
    figlocal = plt.gcf()
 
    #horizontal version
    for s,c in zip(ls,lc):
        text = plt.text(x,y," "+s+" ",color=c, transform=t, **kw)
        text.draw(figlocal.canvas.get_renderer())
        ex = text.get_window_extent()
        t = transforms.offset_copy(text._transform, x=ex.width, units='dots')
 
    plt.show(figlocal) #plt.show((figlocal,fig))

words_from_title = "Соотношение сетевых и несетевых заведений".split()
colors_set = ['black', '#96b2bc', 'black', '#1e78b5', '#1e78b5', 'black']
figure, ax = plt.subplots()
colors = ["#1e78b5", "#96b2bc"]
info_for_graphic = cafe['chain'].value_counts().rename({'нет': 'несетевых', 'да': 'сетевых'})
ax.pie(info_for_graphic.values, autopct='%1.1f%%', startangle=90, colors=colors)
rainbow_text(-1.3, 1.1, words_from_title, colors_set, size=14)
plt.show()

80.7 % заведений являются несетевыми, что говорит о некоторых проблемах в распространении концепций на сеть заведений.

In [ ]:
cafe_chain = cafe.loc[cafe['chain'] == 'да']
plt.figure(figsize=(18,4))
sns.distplot(cafe_chain['number'], bins=20)
plt.xlabel(" ")
plt.ylabel("Плотность")
plt.title("Плотность распределения сетевых ООП по количеству посадочных мест", size=14);

Плотность распределения сетевых ООП по количеству посадочных мест имеет форму распределения Пуассона со смещением влево, т.е. в городе преобладает большее число заведений с малым количеством посадочных мест - до 50 мест, далее плотность распределения идет к резкому уменьшению и практически равна нулю на отметке в 300 посадочных мест.

###2.5. Среднее количество посадочных мест для каждого вида объекта общественного питания

In [ ]:
plt.figure(figsize=(8,4))
ax=sns.barplot(x="object_type", y="number", data=cafe, color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right', size=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.title("Среднее количество посадочных мест для каждого вида ООП", size=18)
plt.xlabel(" ")
plt.ylabel("Количество посадочных мест", size=14);

Среднее количество посадочных мест самое большое в столовых - порядка 130, за ними идут рестораны - 90-100 и буфеты - порядка 50 мест.

Минимальное же количество посадочных мест в отделах кулинарий магазинов и закусочных - в районе 10 мест.

Выделим ТОП-10 сетевых ООП:

In [ ]:
cafe_chain = pd.pivot_table(cafe.query('chain == "да"').loc[:,['object_name', 'number']],
                    values='number', index=['object_name'], aggfunc=['count', 'median', np.mean, np.min, np.max]).reset_index()

cafe_chain.columns = ['object_name', 'count_places', 'median_number', 'mean_number', 'min_number', 'max_number']
cafe_chain['mean_number'] = cafe_chain['mean_number'].astype(np.int64)
cafe_chain['median_number'] = cafe_chain['median_number'].astype(np.int64)
cafe_chain.sort_values('count_places', ascending=False).head(10)

Максимальные по количеству ресторанов сети - Шоколадница, KFC и Макдоналдс.

Среднее количество посадочных мест в ТОП-10 варьируется в пределах от 18 до 87 мест, при этом есть сетевые ООП с количеством мест до 6 шт. - Милти и Суши Wok.

Почти все сети имеют пункты без посадочных мест.

Максимальное количество мест варьируется в диапазоне от 22 до 580 посадочных мест.

###2.6. Выделение в отдельный столбец информацию об улице из столбца address

In [ ]:
words = ['улица','ул','переулок','шоссе','проспект','площадь','проезд',
         'село','аллея','бульвар','набережная','тупик','линия']
str_pat = r".*,\s*\b([^,]*?(?:{})\b[^,]*)[,$]+".format("|".join(words))
cafe["street"] = cafe["address"].str.extract(str_pat, flags=re.I)
cafe.sample(5)

###2.7. Топ-10 улиц по количеству объектов общественного питания

In [ ]:
cafe_streets_top_10=cafe['street'].value_counts().head(10).reset_index()
cafe_streets_top_10.columns = ['street','cafe_number']

In [ ]:
plt.figure(figsize=(10,6))
ax=sns.barplot(x='cafe_number', y="street", data=cafe_streets_top_10, 
                 color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right')
ax.grid(False)
sns.despine(bottom = True, left = True)
for p in ax.patches:
    ax.annotate("%.0f" % p.get_width(), xy=(p.get_width(), p.get_y()+p.get_height()/2),
            xytext=(-5, 0), textcoords='offset points', ha="right", va="center")
ax.axes.get_xaxis().set_visible(False)
plt.xlabel("")
plt.ylabel("")
plt.title("Топ-10 улиц по количеству ООП", size=14);

Для определения района по ТОП-10 улиц воспользуемся внешним ресурсом Яндекс Геокодер по API и выполним следующие действия:

1. Передадим адреса ООП на ТОП-10 улиц и получим их координаты:

In [ ]:
def get_geo_info_about_homes(array_parts_address, api_key, tag):
    adress_list = []

    for address_part in array_parts_address:
        try:

            # В случае надобности координат и районов запросы несколько отличаются
            if tag == 'formatted':
                URL_for_request = 'https://geocode-maps.yandex.ru/1.x/?apikey={}&kind=district&geocode='.format(api_key) + address_part
            else:
                URL_for_request = 'https://geocode-maps.yandex.ru/1.x/?apikey={}&geocode='.format(api_key) + address_part

            data_for_request = requests.get(URL_for_request).text           
            result_requests = re.search(f'<{tag}>(.*?)<', data_for_request).group(1)
            adress_list.append(result_requests)
        except:
            adress_list.append('invalid geo')
            print(address_part)

    return adress_list

In [ ]:
cafe_streets_top_10 = cafe_streets_top_10.street
info_about_popular_places = cafe.query('street in @cafe_streets_top_10')['address'].to_frame()
# info_about_popular_places.loc[:,'coordinates'] = get_geo_info_about_homes(info_about_popular_places['address'].values, api_key, 'pos')
# info_about_popular_places.to_csv('coordinates_popular_places.csv', sep=';', encoding='utf-8-sig')

2. Передадим полученный файл с координатами для получения районов:

In [ ]:
info_about_popular_places=pd.read_csv('/content/drive/MyDrive/Edu/Data Analysis/9_Как рассказать историю с помощью данных/!Project/data/coordinates_popular_places.csv')

In [ ]:
# info_about_popular_places.loc[:,'districts'] = get_geo_info_about_homes(info_about_popular_places['coordinates'].values, api_key, 'formatted')
# info_about_popular_places = info_about_popular_places.drop(['coordinates'], axis=1)
# info_about_popular_places.to_csv('districts_popular_places.csv', sep=';', encoding='utf-8-sig')

3. Обработаем полученные данные:

In [ ]:
districts_popular_places=pd.read_csv('/content/drive/MyDrive/Edu/Data Analysis/9_Как рассказать историю с помощью данных/!Project/data/districts_popular_places.csv', error_bad_lines=False)

In [ ]:
def get_dict_disctricts_and_cut_excess(more_districts):
    dict_districts = { }

    for element_arr in more_districts:
        
        # Блок с try, по причине наличия адресов, для которых не определяются корректно координаты
        try:

            dict_districts[element_arr] = re.match(
                r'(.*?, (поселение [а-яА-Яё ]*),?|.+?, ([а-яА-Яё ]*район[а-яА-Яё ]*,?))', element_arr).group(3)
            if dict_districts[element_arr][-1] == ',':
                dict_districts[element_arr] = dict_districts[element_arr][:-int(dict_districts[element_arr][-1] == ',')]
        except:

            dict_districts[element_arr] = 'invalid'
    return dict_districts

In [ ]:
dict_districts = get_dict_disctricts_and_cut_excess(districts_popular_places['districts'].unique())
districts_popular_places['districts'] = districts_popular_places['districts'].map(dict_districts)
plt.figure(figsize=(10,6))
ax=sns.barplot(x='districts', y="index", data=districts_popular_places['districts'].value_counts().head(10).to_frame().reset_index(), 
                 color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right', size=16)
ax.grid(False)
sns.despine(bottom = True, left = True)
for p in ax.patches:
    ax.annotate("%.0f" % p.get_width(), xy=(p.get_width(), p.get_y()+p.get_height()/2),
            xytext=(-5, 0), textcoords='offset points', ha="right", va="center", size=14)
ax.axes.get_xaxis().set_visible(False)
plt.xlabel("")
plt.ylabel("")
plt.title("Топ-10 районов по максимальному количеству ООП", size=18);

Первые два района по количеству заведений сильно выделяются на фоне остальных, территориально они находятся рядом друг с другом.

Причины большого числа заведений в Пресненском районе:

*  содержит деловой центр «Москва-Сити» — развивающийся деловой район в Москве на Пресненской набережной, в котором большое число организаций и высокая плотность рабочих мест;
*  район включает в себя множество историко-архитектурных памятников;
*  в районе расположены объекты федерального и общегородского значения.
Причины большого числа заведений в Хорошёвском районе:

*  наличие самого большого ТРЦ в Европе -ТЦ Европейский, по официальной информации на сайте, лишь только на его территории находится 81 ресторан;
*  наличие крупного стадиона в районе - ЦСКА.

###2.8. Число улиц с одним объектом общественного питания

In [ ]:
streets_with_one_cafe = cafe.groupby(['street'])['id'].nunique().\
            to_frame().query('id == 1').index
print(f'В Москве {len(streets_with_one_cafe)} улица с одним ООП')

Определим районы этих улиц:

In [ ]:
# one_type_institute_df = cafe.query('street in @streets_with_one_cafe')['address'].to_frame()
# one_type_institute_df.loc[:,'coordinates'] = get_geo_info_about_homes(one_type_institute_df['address'].values, api_key, 'pos')
# one_type_institute_df = one_type_institute_df.query('coordinates != "invalid geo"')

# one_type_institute_df.loc[:, 'districts'] = get_geo_info_about_homes(one_type_institute_df['coordinates'].values, api_key, 'formatted')
# one_type_institute_df = one_type_institute_df.query('districts != "invalid geo"')
# one_type_institute_df = one_type_institute_df.drop(['coordinates', 'address'], axis=1)
# one_type_institute_df.to_csv('districts_one_type_institute.csv', sep=';', encoding='utf-8-sig')

In [ ]:
one_type_institute_df=pd.read_csv('/content/drive/MyDrive/Edu/Data Analysis/9_Как рассказать историю с помощью данных/!Project/data/districts_one_type_institute.csv', error_bad_lines=False)

In [ ]:
dict_districts = get_dict_disctricts_and_cut_excess(one_type_institute_df['districts'].unique())
one_type_institute_df['districts'] = one_type_institute_df['districts'].map(dict_districts)
one_type_institute_df.drop(one_type_institute_df[one_type_institute_df['districts']=='invalid'].index, inplace=True)
one_type_institute_df['districts'].value_counts().head(10).reset_index()                                                      
plt.figure(figsize=(10,6))
ax=sns.barplot(x='districts', y="index", data=one_type_institute_df['districts'].value_counts().head(10).to_frame().reset_index(), 
                 color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right')
ax.grid(False)
sns.despine(bottom = True, left = True)
for p in ax.patches:
    ax.annotate("%.0f" % p.get_width(), xy=(p.get_width(), p.get_y()+p.get_height()/2),
            xytext=(-5, 0), textcoords='offset points', ha="right", va="center")
ax.axes.get_xaxis().set_visible(False)
plt.xlabel("")
plt.ylabel("")
plt.title("Топ-10 районов по количеству улиц с одним ООП", size=14);

На 551 улице располагается один ООП.

Наличие одного ОПП на улице связано с инфраструктурой и наличием спроса в конкретном микрорайоне, а также размером самой улицы, так, например, маленькие переулки в центре города.

В перечень районов также попали районы из ТОП-10 по максимальному количеству ОПП.

При этом не проанализированными остаются улицы города, на которых нет ООП вовсе, поэтому считаю этот показатель малоэффективным для рассмотрения и последующего прогнозирования.

###2.9. Распределение количества посадочных мест для улиц с большим количеством объектов общественного питания

In [ ]:
number_by_street = cafe.query('street in @cafe_streets_top_10').groupby('street')['number'].\
        agg('sum').reset_index()

In [ ]:
plt.figure(figsize=(10,6))
ax=sns.barplot(x='number', y="street", data=number_by_street, 
            order=number_by_street.sort_values('number', ascending = False).street,
                 color="salmon")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, horizontalalignment ='right')
ax.grid(False)
sns.despine(bottom = True, left = True)
for p in ax.patches:
    ax.annotate("%.0f" % p.get_width(), xy=(p.get_width(), p.get_y()+p.get_height()/2),
            xytext=(-5, 0), textcoords='offset points', ha="right", va="center")
ax.axes.get_xaxis().set_visible(False)
plt.xlabel("")
plt.ylabel("")
plt.title("Топ-10 районов по количеству улиц с одним ООП", size=14);

На проспекте Мира почти 13 тыс. посадочных мест для посетителей.

На остальных улицах в среднем от 8 до 9,5 тыс. мест.

Замыкает ТОП-10 Каширское шоссе, на котором посадочных мест более чем в 2 раза меньше чем на проспекте Мира - 6,2 тыс.

##Общий вывод
---

В Москве из всех видов ООП самыми многочисленными являются кафе, при этом столовых и ресторанов почти в 3 раза меньше, т.е. конкуренция в этих видах ООП намного ниже, а популярность их высока.

Соотношение несетевых к сетевым заведениям по всем видам ООП достаточно большое и в среднем доля сетевых ООП составляет 19,3 %. Среди сетевых ООП минимальная доля приходится на столовые - всего 0,12 %.

Количество посадочных мест очень разное в зависимости от вида ООП: самое большое в столовых - порядка 130, за ними идут рестораны - 90-100 и буфеты - порядка 50 мест, минимальное же количество посадочных мест в отделах кулинарии магазинов и закусочных - в районе 10 мест.
При этом в ТОП-10 сетевых ООП по количеству заведений количество мест варьируется в диапазоне 18 до 87 мест.

Почти все сети имеют пункты без посадочных мест.
Максимальное количество мест варьируется в диапазоне от 22 до 580 посадочных мест.

По районам можно в первую очередь ориентироваться на самые популярные районы по ОПП, но также требуется провести дополнительный анализ для определения объектов инфраструктуры посетители которых могут быть потенциальными клиентами нового заведения.

В силу того, что заведение планируется обслуживать роботами, то целесообразно выбрать формат сетевых заведений, т.к. он является наиболее оптимальным для тиражирования стандартов во всей сети заведений.

В связи с вышеизложенным основными вариантами развития представляются следующие:

<strong>открытие сетевых столовых:</strong>
<ul>
<li>плюсы:
<ul>
<li>самая низкая доля среди всех ООП;</li>
<li>большое количество потенциально доступных районов для открытия заведений;</li>
<li>несложная логистика для роботов обслуживающее заведение.</li>
</ul>
</li>
<li>минусы:
<ul>
<li>большое количество посадочных мест.</li>
</ul>
</ul>

<strong>открытие сетевых ресторанов:</strong>
<ul>
<li>плюсы:
<ul>
<li>составляют всего четверть рынка среди сетевых ОПП, а по отношение к сетевым ресторанам сетевых в 3 раза меньше;</li>
<li>большое количество потенциально доступных районов для открытия заведений;</li>
<li>количество посадочных мест может быть различным.</li>
</ul>
</li>
<li>минусы:
<ul>
<li>сложная логистика для роботов обслуживающее заведение.</li>
</ul>
</ul>

<strong>открытие сетевых бистро:</strong>
<li>плюсы:
<ul>
<li>большое количество потенциально доступных районов для открытия заведений;</li>
<li>количество посадочных мест может быть различным;</li>
<li>несложная логистика для роботов обслуживающее заведение.</li>
</ul>
</li>
<li>минусы:
<ul>
<li>высокая конкуренция на рынке.</li>
</ul>
</ul>

[Презентация](https://docs.google.com/presentation/d/1mtLknPq7vTkbE60C0Bt3LP5bXq0PPVGO_trReLJ0P2U/edit?usp=sharing)